In [15]:
#importing libraries
import numpy as np
import fetchmaker
from scipy.stats import binom_test
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import chi2_contingency

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm



The attributes that FetchMaker keeps track of are:

weight, an integer representing how heavy a dog is in pounds
tail_length, a float representing tail length in inches
age, in years
color, a String such as "brown" or "grey"
is_rescue, a boolean 0 or 1
The fetchmaker package lets you access this data for a specific breed of dog with the following format:

fetchmaker.get_weight("poodle")
This returns a Pandas DataFrame of the weights of the poodles recorded in the system. The other methods are get_tail_length, get_color, get_age, and get_is_rescue, which all take a breed as an input.

In [17]:
#Get the tail lengths of all of the "rottweiler"s in the system, and store it in a variable called rottweiler_tl.
rottweiler_tl =fetchmaker.get_tail_length('rottweiler')
print(rottweiler_tl)

400    3.13
401    3.32
402    1.16
403    2.23
404    8.86
       ... 
495    3.28
496    5.50
497    2.95
498    4.64
499    3.39
Name: tail_length, Length: 100, dtype: float64


In [18]:
#Print out the mean of rottweiler_tl and the standard deviation of rottweiler_tl
rottweiler_tl_mean = np.mean(rottweiler_tl)
rottweiler_tl_std = np.std(rottweiler_tl)
print('The rottweiler avg tail length is {} \n'. format(rottweiler_tl_mean))
print('The rottweiler std dev of tail length is {} \n'. format(rottweiler_tl_std))


The rottweiler avg tail length is 4.2361 

The rottweiler std dev of tail length is 2.0647536874891395 



#Data to the rescue



Over the years, we have seen that we expect 8% of dogs in the FetchMaker system to be rescues. We want to know if whippets are significantly more or less likely to be a rescue.

In [19]:
whippet_rescue = fetchmaker.get_is_rescue('whippet')
print(whippet_rescue)

700    0
701    0
702    0
703    0
704    0
      ..
795    0
796    0
797    0
798    0
799    0
Name: is_rescue, Length: 100, dtype: int64


Use np.count_nonzero to get the number of entries in whippet_rescue that are 1

In [21]:
num_whippet_rescues = np.count_nonzero(whippet_rescue)
print('The count of (1) entry in the whippet_rescue is {} \n'.format(num_whippet_rescues))


The count of (1) entry in the whippet_rescue is 6 



Get the number of samples in the whippet set by taking the np.size of whippet_rescue. Store this in a variable called num_whippets.

In [23]:

num_whippets = np.size(whippet_rescue)
print('The number of samples in the whippet_rescue is {} \n'.format(num_whippets))


The number of samples in the whippet_rescue is 100 




Use a binomial test to test the number of whippet rescues, num_whippet_rescues, against our expected percentage, 8%.

In [26]:
binom_test_whippets_rescues = binom_test(num_whippet_rescues,num_whippets,0.8)
print('The P-Value of the whippet_rescue is {} \n'.format(binom_test_whippets_rescues))
print('So the P-Value from the Whippet_Rescue Binomial Test is %.3f and therefore, we accept the null hypothesis, which is that there is no difference between the observed number of whippet rescues and our expected whippet rescues percentage'%(binom_test_whippets_rescues))
print('\n')

The P-Value of the whippet_rescue is 6.28850444995885e-58 

So the P-Value from the Whippet_Rescue Binomial Test is 0.000 and therefore, we accept the null hypothesis, which is that there is no difference between the observed number of whippet rescues and our expected whippet rescues percentage




Three of our most popular mid-sized dog breeds are whippets, terriers, and pitbulls. Is there a significant difference in the average weights of these three dog breeds? Perform a comparative numerical test to determine if there is a significant difference.

In [30]:
w = fetchmaker.get_weight('whippet')
t = fetchmaker.get_weight('terrier')
p = fetchmaker.get_weight('pitbull')

ANOVA_mid_size_dogs = f_oneway(w, t, p)
print('The P-value obtained from the ANOVA test on these three popular breeds is %.10f and therefore, we reject the null hypothesis, which is there is significant difference in the average weights of these three dogs, but we do not know which pair of datasets is significantly different.'% (ANOVA_mid_size_dogs[1]))
print('\n')

The P-value obtained from the ANOVA test on these three popular breeds is 0.0000000000 and therefore, we reject the null hypothesis, which is there is significant difference in the average weights of these three dogs, but we do not know which pair of datasets is significantly different.





Now, perform another test to determine which of the pairs of these dog breeds differ from each other.

In [32]:
values = np.concatenate([w , t , p])
labels = ['whippet'] *len(w) + ['terrier'] *len(t) + ['pitbull'] * len(p)
tukey_result = pairwise_tukeyhsd(values, labels, alpha = 0.05)

print("Below is the table generated from the Tukey's Range Test to find out which pair of datasets is statistically different: \n {}".format(tukey_result))
print('\n')

Below is the table generated from the Tukey's Range Test to find out which pair of datasets is statistically different: 
  Multiple Comparison of Means - Tukey HSD, FWER=0.05 
 group1  group2 meandiff p-adj   lower  upper  reject
-----------------------------------------------------
pitbull terrier   -13.24  0.001 -16.728 -9.752   True
pitbull whippet    -3.34 0.0639  -6.828  0.148  False
terrier whippet      9.9  0.001   6.412 13.388   True
-----------------------------------------------------




#Categorical dog test



We want to see if "poodle"s and "shihtzu"s have significantly different color breakdowns.

Get the poodle colors and store it in a variable called poodle_colors.

Get the shih tzu colors and store it in a variable called shihtzu_colors

In [33]:
poodle_colors = fetchmaker.get_color('poodle')
shihtzu_colors = fetchmaker.get_color('shihtzu')

In [34]:
black_poodle = np.count_nonzero(poodle_colors == 'black')
brown_poodle = np.count_nonzero(poodle_colors == 'brown')
gold_poodle = np.count_nonzero(poodle_colors == 'gold')
grey_poodle = np.count_nonzero(poodle_colors == 'grey')
white_poodle = np.count_nonzero(poodle_colors == 'white')
#Secondly, obtain the color numbers for shihtzu breed
black_shihtzu = np.count_nonzero(shihtzu_colors == 'black')
brown_shihtzu = np.count_nonzero(shihtzu_colors == 'brown')
gold_shihtzu = np.count_nonzero(shihtzu_colors == 'gold')
grey_shihtzu = np.count_nonzero(shihtzu_colors == 'grey')
white_shihtzu = np.count_nonzero(shihtzu_colors == 'white')
#Next, create the contingency table using a list of lists
color_table = [[black_poodle, black_shihtzu],[brown_poodle, brown_shihtzu], [gold_poodle, gold_shihtzu], [grey_poodle, grey_shihtzu], [white_poodle, white_shihtzu]]

chi2, pval, dof, expected = chi2_contingency(color_table)
print('The statistic of the color_table dataset is %.3f \n'%(chi2))
print('The P-Value of the color_table dataset is %.3f \n'% (pval))
print('The degrees of freedom from the color_table dataset is {} \n'.format(dof))
print('The expected table is as follows: \n {}'.format(expected))
print('\n')
print('The conclusion from the Chi-Square test above is since the P-Value is %.3F, we reject the null hypothesis and stated that there is a significant difference between the datasets'% (pval))

The statistic of the color_table dataset is 14.727 

The P-Value of the color_table dataset is 0.005 

The degrees of freedom from the color_table dataset is 4 

The expected table is as follows: 
 [[13.5 13.5]
 [24.5 24.5]
 [ 7.   7. ]
 [46.5 46.5]
 [ 8.5  8.5]]


The conclusion from the Chi-Square test above is since the P-Value is 0.005, we reject the null hypothesis and stated that there is a significant difference between the datasets
